In [ ]:
# default_exp data.manipulate

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import numpy as np
import pandas as pd
import pydicom

# Intelligent Imaging Data Dongle

> Suite of python tools for handling medical data.

## DataLoader

In [ ]:
#export
def dcm2array(path_to_dicom_dir, sort_by_slice_location=True):
    """
    Transform DICOM data into numpy array.
    
    Parameters
    ----------
    path_to_dicom_dir : str
        path to folder containing all dicom files for *one patient*
    sort_by_slice_location : bool
        if True, returning array will be ordered by slice location

    Returns
    -------
    pixel_array : arr
        3D array containing pixel data from dicom files
        
    Notes
    -------
    This function will require GDCM to be already installed
    if the DICOM is compressed
    
    """
    try:
        import gdcm                     
    
    except ImportError:
        print("GDCM needs to be installed.")
        print("Try: conda install -c conda-forge gdcm")

    else:  
        df = pd.DataFrame()
        df['filename']= os.listdir(path_to_dicom_dir)
        df['pathname']= path_to_dicom_dir + df['filename']

        df['DS']=[pydicom.dcmread(x) for x in df['pathname']]
        df['SOPInstanceUID'] = [x.SOPInstanceUID for x in df['DS']]
        df['SliceLoc'] = [x.InstanceNumber for x in df['DS']]
        df['Pixels'] = [x.pixel_array for x in df['DS']]
        tempo = df['DS'][0]
        im_at = tempo.AcquisitionMatrix, tempo.PixelSpacing

        if sort_by_slice_location == True:
            df = df.sort_values(by=['SliceLoc'])
        pixel_array = np.dstack(np.asarray(df['Pixels']))

        return pixel_array

In [ ]:
# example usage:
import matplotlib.pyplot as plt
data_path = '../data/knee/'
try:
    array = dcm2array(path_to_dicom_dir = data_path, sort_by_slice_location=True)
    plt.imshow(array[:,:,0])
    plt.show()
except:
    print('could not load array')

GDCM needs to be installed.
Try: conda install -c conda-forge gdcm
could not load array
